<a href="https://colab.research.google.com/github/AndrewDavidRatnam/Decision_Trees/blob/main/Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

## Training a Decision Tree on the Moons Dataset

In [4]:
X_moons, y_moons = make_moons(n_samples=10_000, noise=0.5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons, test_size=0.2,
                                                    random_state=42)

In [5]:
X_train.max(), X_train.min()
# no need to use standard Scaler as decision is invariant to scale as a whole gini index takes ratio
# also make moons range is not that great, if you range is extremly high or low , to ease computation may be used

(3.6401170138545806, -2.733401526268456)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# params = {
#     "max_leaf_nodes": list(range(2, 100)), #98 values
#     "max_depth": list(range(1, 7)), #6 values
#     "min_samples_split": [2, 3, 4] # 3values    3*6*98 grid search wheww
# }

# grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1,
#                               cv=3)
# grid_search_cv.fit(X_train, y_train)
decision_tree_clf = DecisionTreeClassifier(max_depth=6, max_leaf_nodes=17,random_state=42)
decision_tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6, max_leaf_nodes=17, random_state=42)

In [10]:
from sklearn.metrics import accuracy_score

# y_pred = grid_search_cv.predict(X_test)
y_pred = decision_tree_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.831

{'max_depth': 2, 'max_leaf_nodes': 4, 'min_samples_split': 2}

In [11]:
#1000 subsets of training data each containing 100 instances selected randomly

from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances =100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)

for mini_train_index, mini_test_index in rs.split(X_train):
  X_mini_train = X_train[mini_train_index]
  y_mini_train = y_train[mini_train_index]
  mini_sets.append((X_mini_train, y_mini_train))

In [15]:
#training one decision tree on each subset using hyper parameters above in random_search cv
from sklearn.base import clone

forest = [clone(decision_tree_clf) for _ in range(n_trees)]
accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
  tree.fit(X_mini_train, y_mini_train)

  y_pred = tree.predict(X_test)
  accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.7567535000000001

In [16]:
# generate predictions for the test set from all the trees and use the mode of the predictied value
from scipy.stats import mode
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
  Y_pred[tree_index] = tree.predict(X_test)

y_pred_majority_votes , n_votes = mode(Y_pred, axis=0)
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.832

In [19]:
((accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))-np.mean(accuracy_scores))/np.mean(accuracy_scores))*100 #9% increase

9.943330291832133